# Climbing Data Analysis Pipeline

This notebook processes and analyzes climbing data exported from Mountain Project to extract meaningful insights about climbing patterns, progression, and performance.

## Data Overview
The input data consists of climbing "ticks" (recorded climbs) from [MountainProject](https://www.mountainproject.com/)containing information about:
- Routes climbed (name, grade, style)
- Location details (area, crag)
- Performance metrics (send type, attempts)
- Personal ratings and comments

## Pipeline Structure

### 1. Data Loading & Cleaning
The `load_and_clean_data()` function performs initial data processing:
- Converts dates to datetime format
- Extracts countries from location strings
- Creates binary columns for climb types (Sport, Trad, Boulder)
- Creates binary columns for send styles (Onsight, Flash, Redpoint)

### 2. Feature Engineering
The `add_enhanced_features()` function enriches the data with:
- Temporal features (Year, Month, Season)
- Weekend/Weekday classification
- Success metrics
- Project tracking (attempts, completion status)
- Location hierarchy parsing (Area, Subarea, Crag)
- Grade categorization (5.9 and under, 5.10, 5.11, etc.)

### 3. Data Type Optimization
The `convert_to_categorical()` function:
- Converts appropriate columns to categorical data type
- Reduces memory usage
- Improves performance for analysis

### 4. Metrics & Analysis Functions
Several specialized functions provide key insights:

#### Comprehensive Metrics
`analyze_climbing_metrics()` calculates:
- Overall climbing volume and unique routes
- Success rates by type and grade
- Project statistics
- Location diversity
- Temporal patterns
- Grade distributions
- Star ratings analysis

#### Progression Tracking
`get_hardest_climb_by_week()` tracks:
- Weekly maximum grades
- Send styles
- Volume of climbing
- Separate analysis for bouldering and rope climbing

#### Location Analysis
`get_ticks_by_crag()` and `get_ticks_by_country()` analyze:
- Most frequented areas
- Geographic distribution of climbs
- International vs domestic climbing

## Usage
The notebook processes climbing data through these steps in sequence, creating a rich dataset ready for visualization in Tableau or other analysis tools.

The final output includes:
- Processed data with all engineered features
- Summary metrics for overall performance
- Weekly progression tracking
- Location distribution analysis

In [1]:
import datetime as dt
import numpy as np
import pandas as pd

##Functions

In [2]:
def extract_country(location):
    """
    Extract country from location string.
    Assumes US if location starts with a state, otherwise parses international format.

    Args:
        location (str): Full location string from Mountain Project

    Returns:
        str: Country name
    """
    if pd.isna(location):
        return None

    # Split location into parts
    parts = [part.strip() for part in location.split('>')]

    # Check if it's an international location
    if parts[0].lower() == 'international':
        # International format: "International > Continent > Country > ..."
        if len(parts) >= 3:
            return parts[2]
        return None
    else:
        # If it starts with a state, it's in the US
        return "United States"

In [3]:
def get_state(row):
    """Extract state information from Location for US entries"""
    if row['Country'] == 'United States':
        # Split by '>' and take first part, strip whitespace
        return row['Location'].split('>')[0].strip()
    return 'Not State'

In [4]:
def load_and_clean_data(data):
    """
    Load and clean climbing data from CSV format.

    Args:
        data (str): Path to CSV file containing climbing data

    Returns:
        pandas.DataFrame: Cleaned DataFrame with additional engineered features including:
            - Datetime conversion for dates
            - Numeric grade extraction
            - Binary columns for send types (Onsight, Flash, Redpoint, etc.)
            - Binary columns for route types (Sport, Trad, Boulder)
            - Binary column for multipitch routes
    """
    # Convert data to DataFrame
    df = pd.read_csv(data)

    # Convert date to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Clean up Rating Code to numeric
    df['Rating Code'] = pd.to_numeric(df['Rating Code'])

    # Set a column as country
    df['Country'] = df['Location'].apply(extract_country)
    df['State'] = df.apply(get_state, axis=1)

    # Extract numeric grade from Rating when possible
    df['Numeric Grade'] = df['Rating'].str.extract(r'5\.(\d+[a-d]?)').fillna(df['Rating'])

    # Create style columns using Lead Style and Style columns
    df['Is_Onsight'] = df['Lead Style'].str.contains('onsight', case=False, na=False)
    df['Is_Flash'] = df['Style'].str.contains('flash', case=False, na=False)
    df['Is_Redpoint'] = df['Lead Style'].str.contains('redpoint|send', case=False, na=False)
    df['Is_Fell_Hung'] = df['Lead Style'].str.contains('fell/hung', case=False, na=False)

    # Calculate metrics for route types
    df['Is_Sport'] = df['Route Type'].str.contains('Sport', case=False, na=False)
    df['Is_Trad'] = df['Route Type'].str.contains('Trad', case=False, na=False)
    df['Is_Boulder'] = df['Route Type'].str.contains('Boulder', case=False, na=False)
    df['Is_Multipitch'] = df['Pitches'] > 1

    # Remove unneeded data
    df = df.drop(columns=['URL'])

    return df

In [5]:
def add_enhanced_features(df):
    """
    Add enhanced features to the climbing dataframe for more detailed analysis.

    Args:
        df (pandas.DataFrame): Input DataFrame with climbing data

    Returns:
        pandas.DataFrame: DataFrame with additional features
    """
    # Create a copy to avoid modifying the original
    enhanced_df = df.copy()

    # Add temporal features
    enhanced_df['Year'] = enhanced_df['Date'].dt.year
    enhanced_df['Month'] = enhanced_df['Date'].dt.month
    enhanced_df['Season'] = enhanced_df['Date'].dt.month.map({
        12: 'Winter', 1: 'Winter', 2: 'Winter',
        3: 'Spring', 4: 'Spring', 5: 'Spring',
        6: 'Summer', 7: 'Summer', 8: 'Summer',
        9: 'Fall', 10: 'Fall', 11: 'Fall'
    })
    enhanced_df['Is_Weekend'] = enhanced_df['Date'].dt.dayofweek.isin([5, 6])

    # Create session identifier for multiple attempts on same route
    enhanced_df['Session_ID'] = (
        enhanced_df.groupby(['Route', 'Date'])
        .cumcount()
        .add(1)
    )

    # Calculate number of days attempted for each route
    route_attempts = (
        enhanced_df.groupby('Route')
        .agg({
            'Date': 'nunique'
        })
        .rename(columns={'Date': 'Days_Attempted'})
    )

    # Add days attempted to main dataframe
    enhanced_df = enhanced_df.merge(
        route_attempts,
        left_on='Route',
        right_index=True,
        how='left'
    )

    # Add first try and redpoint status on a per-entry basis
    enhanced_df['Was_First_Try'] = (
        (enhanced_df['Style'].str.contains('Onsight|Flash', case=False, na=False)) |
        (enhanced_df['Lead Style'].str.contains('Onsight|Flash', case=False, na=False))
    )

    # Get first attempt date for each route
    first_attempts = enhanced_df.groupby('Route')['Date'].min().reset_index()
    first_attempts.columns = ['Route', 'First_Attempt_Date']

    # Find successful attempts
    enhanced_df['Is_Success'] = (
        (enhanced_df['Style'].str.contains('Send|Redpoint|Flash|Onsight',
                                       case=False,
                                       na=False)) |
        (enhanced_df['Lead Style'].str.contains('Redpoint|Flash|Onsight|Send',
                                            case=False,
                                            na=False))
    )

    # Join first attempt dates back to main DataFrame
    enhanced_df = enhanced_df.merge(first_attempts, on='Route', how='left')

    # Find routes where success happened on first day
    success_on_first_day = (
        enhanced_df[
            (enhanced_df['Date'] == enhanced_df['First_Attempt_Date']) &
            enhanced_df['Is_Success']
        ]['Route'].unique()
    )

    # Mark projects - routes with multiple days AND no success on first day
    enhanced_df['Is_Project'] = (
        (enhanced_df['Days_Attempted'] > 1) &
        ~enhanced_df['Route'].isin(success_on_first_day)
    )

    # Clean up
    enhanced_df = enhanced_df.drop(columns=['First_Attempt_Date'])

    # Clean up notes and extract length if missing
    enhanced_df['Notes'] = (
        enhanced_df['Notes']
        .str.replace('&#39;', "'")
        .str.replace('&quot;', '"')
        .fillna('')
    )

    # Parse location hierarchy
    def get_crag(location):
        if pd.isna(location):
            return None
        parts = location.split(' > ')
        return parts[-1] if parts else None

    enhanced_df['Crag'] = enhanced_df['Location'].apply(get_crag)

    location_parts = enhanced_df['Location'].str.split(' > ', expand=True)
    if location_parts.shape[1] >= 4:  # Ensure we have enough parts
        enhanced_df['Area'] = location_parts[2]
        enhanced_df['Subarea'] = location_parts[3]

    # Calculate success metrics
    enhanced_df['Is_Success'] = (
        (enhanced_df['Style'].str.contains('Send|Redpoint|Flash|Onsight',
                                       case=False,
                                       na=False)) |
        (enhanced_df['Lead Style'].str.contains('Redpoint|Flash|Onsight',
                                            case=False,
                                            na=False))
    )

    # Add grade category
    def categorize_grade(grade):
        if pd.isna(grade):
            return 'Unknown'
        if 'V' in str(grade):
            return 'Boulder'
        grade = str(grade).replace('-', '').replace('+', '')
        if '5.13' in grade:
            return '5.13'
        elif '5.12' in grade:
            return '5.12'
        elif '5.11' in grade:
            return '5.11'
        elif '5.10' in grade:
            return '5.10'
        else:
            return '5.9 and under'

    enhanced_df['Grade_Category'] = enhanced_df['Rating'].apply(categorize_grade)

    return enhanced_df

In [6]:
def convert_to_categorical(df, columns_list):
    """
    Convert specified columns in a DataFrame to categorical data type.

    Args:
        df (pandas.DataFrame): Input DataFrame
        columns_list (list): List of column names to convert to categorical

    Returns:
        pandas.DataFrame: DataFrame with specified columns converted to categorical
    """
    # Create a copy of the DataFrame to avoid modifying the original
    df_copy = df.copy()

    # Convert each specified column to categorical
    for column in columns_list:
        if column in df_copy.columns:
            df_copy[column] = df_copy[column].astype('category')
        else:
            print(f"Warning: Column '{column}' not found in DataFrame")

    return df_copy

In [7]:
def analyze_climbing_metrics(df):
    """
    Calculate comprehensive metrics from climbing data.

    Args:
        df (pandas.DataFrame): Cleaned climbing DataFrame with categorical data

    Returns:
        dict: Dictionary containing detailed climbing metrics
    """
    # Get indexes of max grades by style

    # For sport climbing
    sport_max_idx = df[df['Is_Sport']]['Rating Code'].idxmax()
    sport_clean_max_idx = df[(df['Is_Sport']) & (df['Is_Success'])]['Rating Code'].idxmax()

    # For trad climbing
    trad_max_idx = df[df['Is_Trad']]['Rating Code'].idxmax()
    trad_clean_max_idx = df[(df['Is_Trad']) & (df['Is_Success'])]['Rating Code'].idxmax()

    # For bouldering
    boulder_max_idx = df[df['Is_Boulder']]['Rating Code'].idxmax()
    boulder_clean_max_idx = df[(df['Is_Boulder']) & (df['Is_Success'])]['Rating Code'].idxmax()

    metrics = {
        # Overall metrics
        'total_climbs': len(df),
        'unique_routes': df['Route'].nunique(),
        'avg_stars': df['Your Stars'].mean(),
        'total_pitches': df['Pitches'].sum(),

        # Climbing type counts
        'sport_climbs': df['Is_Sport'].sum(),
        'trad_climbs': df['Is_Trad'].sum(),
        'boulder_problems': df['Is_Boulder'].sum(),
        'multipitch_climbs': df['Is_Multipitch'].sum(),

        # Success metrics by type
        'sport_success_rate': (df[df['Is_Sport']]['Is_Success'].mean() * 100).round(1),
        'trad_success_rate': (df[df['Is_Trad']]['Is_Success'].mean() * 100).round(1),
        'boulder_success_rate': (df[df['Is_Boulder']]['Is_Success'].mean() * 100).round(1),

        # Send style counts
        'onsight_count': df['Is_Onsight'].sum(),
        'flash_count': df['Is_Flash'].sum(),
        'redpoint_count': df['Is_Redpoint'].sum(),
        'fell_hung_count': df['Is_Fell_Hung'].sum(),

        # Project metrics
        'total_projects': df['Is_Project'].sum(),
        'average_attempts_per_route': df['Days_Attempted'].mean().round(1),

        # Grade metrics
        'hardest_sport': df.loc[sport_max_idx, 'Rating'],
        'hardest_sport_clean': df.loc[sport_clean_max_idx, 'Rating'],
        'hardest_trad': df.loc[trad_max_idx, 'Rating'],
        'hardest_trad_clean': df.loc[trad_clean_max_idx, 'Rating'],
        'hardest_boulder': df.loc[boulder_max_idx, 'Rating'],
        'hardest_boulder_clean': df.loc[boulder_clean_max_idx, 'Rating'],

        # Location metrics
        'unique_areas': df['Area'].nunique(),
        'unique_crags': df['Crag'].nunique(),
        'most_visited_area': df['Area'].value_counts().index[0],
        'most_visited_crag': df['Crag'].value_counts().index[0],

        # Grade distribution
        'grade_distribution': df['Grade_Category'].value_counts().to_dict(),

        # Time-based metrics
        'climbing_days': df['Date'].nunique(),
        'weekend_ratio': (df['Is_Weekend'].mean() * 100).round(1),

        # Season distribution
        'season_distribution': df['Season'].value_counts().to_dict(),

        # Success metrics
        'overall_success_rate': (df['Is_Success'].mean() * 100).round(1),
        'first_try_success_rate': (df['Was_First_Try'].mean() * 100).round(1),

        # Country metrics
        'countries_visited': df['Country'].nunique(),
        'climbs_by_country': df['Country'].value_counts().to_dict()
    }

    # Add average stars by grade category
    metrics['avg_stars_by_grade'] = df.groupby('Grade_Category')['Your Stars'].mean().round(2).to_dict()

    # Add success rates by grade category
    metrics['success_rate_by_grade'] = (
        df.groupby('Grade_Category')['Is_Success']
        .agg(lambda x: (x.mean() * 100).round(1))
        .to_dict()
    )

    return metrics

In [8]:
def get_hardest_climb_by_week(df):
    """
    Analyze grade progression over time for different climbing types.
    Returns a DataFrame with max grades by week, including the route name,
    actual grade, and lead style of the hardest climb.

    Args:
        df (pandas.DataFrame): Input climbing data DataFrame

    Returns:
        pandas.DataFrame: DataFrame with weekly maximums and associated climb details
    """
    # For each date, get the row with the maximum grade
    daily_max_idx = df.groupby('Date')['Rating Code'].idxmax()
    daily_max_data = df.loc[daily_max_idx]

    # Resample by week and aggregate
    weekly_max = daily_max_data.resample('W', on='Date').agg({
        'Rating Code': 'max',
        'Route Type': 'count',
        'Route': 'last',  # Gets the route name for the hardest climb
        'Rating': 'last', # Gets the actual grade
        'Lead Style': 'last'  # Gets the style (e.g., onsight, redpoint)
    }).reset_index()

    # Rename columns for clarity
    weekly_max.columns = ['Week', 'Max_Grade', 'Number_of_Climbs',
                         'Hardest_Route', 'Grade', 'Style']
    weekly_max = weekly_max.drop(columns=['Max_Grade'])
    return weekly_max

In [9]:
def get_ticks_by_crag(df):
    """
    Analyze climbing locations and frequency.
    Returns a Series with location counts.
    """
    location_counts = df['Crag'].value_counts()

    return location_counts

In [10]:
def get_ticks_by_country(df):
    """
    Analyze climbing locations and frequency.
    Returns a Series with location counts.
    """
    location_counts = df['Country'].value_counts()

    return location_counts

In [11]:
def clean_grade(grade):
    """Clean climbing grades to basic form."""
    if pd.isna(grade) or 'snow' in str(grade).lower():
        return None

    grade = str(grade)
    if not grade.startswith('5.'):
        return None

    # Extract basic grade
    parts = grade.split('.')
    if len(parts) != 2:
        return None

    base = parts[1]

    # Handle different grade formats
    if len(base) >= 2:
        # Extract numeric part (5.10, 5.11, etc)
        num = ''.join(filter(str.isdigit, base[:2]))

        # Extract letter grade if it exists
        letter = None
        for char in base:
            if char in 'abcd':
                letter = char
                break

        if letter:
            return f"5.{num}{letter}"
        return f"5.{num}"

    # Handle single digit grades (5.9, 5.8, etc)
    num = base[0]
    return f"5.{num}"

##Read in Data & Initial Investigaton

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
df = load_and_clean_data("/content/drive/My Drive/Colab Notebooks/personal_projects/climbing_analysis/ticks.csv")
df = add_enhanced_features(df)

### View Data

In [14]:
df.head(10)

,Date,Route,Rating,Notes,Pitches,Location,Avg Stars,Your Stars,Style,Lead Style,...,Is_Weekend,Session_ID,Days_Attempted,Was_First_Try,Is_Success,Is_Project,Crag,Area,Subarea,Grade_Category
0,2024-12-24,"Fear and Loathing, III",5.12a,Absolutely 0 left in the tank. I really actual...,1,Nevada > Southern Nevada > Red Rocks > (04) Se...,3.6,2,Lead,Fell/Hung,...,False,1,1,False,False,False,Wall of Confusion,Red Rocks,(04) Second Pullout (Calico II),5.12
1,2024-12-24,Desert Pickle,5.11c,40th pitch on day 4,1,Nevada > Southern Nevada > Red Rocks > (04) Se...,2.8,2,Lead,Onsight,...,False,1,1,True,True,False,Wall of Confusion,Red Rocks,(04) Second Pullout (Calico II),5.11
2,2024-12-24,American Sportsman,5.10b,,1,Nevada > Southern Nevada > Red Rocks > (04) Se...,2.6,2,Lead,Onsight,...,False,1,1,True,True,False,Wall of Confusion,Red Rocks,(04) Second Pullout (Calico II),5.10
3,2024-12-24,The Runaway,5.10b,,1,Nevada > Southern Nevada > Red Rocks > (04) Se...,2.2,1,Lead,Redpoint,...,False,1,2,False,True,False,Wall of Confusion,Red Rocks,(04) Second Pullout (Calico II),5.10
4,2024-12-23,The Prophet,5.12b,37th pitch in 3 days. I CANNOT believe it! Glo...,1,Nevada > Southern Nevada > Red Rocks > (01) Ca...,3.7,4,Lead,Redpoint,...,False,1,1,False,True,False,Alternative Crag,Red Rocks,(01) Calico Basin,5.12
5,2024-12-23,The Prophet,5.12b,,1,Nevada > Southern Nevada > Red Rocks > (01) Ca...,3.7,4,Lead,Fell/Hung,...,False,2,1,False,False,False,Alternative Crag,Red Rocks,(01) Calico Basin,5.12
6,2024-12-23,The Prophet,5.12b,Warmup. VERY FUN!,1,Nevada > Southern Nevada > Red Rocks > (01) Ca...,3.7,4,Lead,Fell/Hung,...,False,3,1,False,False,False,Alternative Crag,Red Rocks,(01) Calico Basin,5.12
7,2024-12-22,Patient Zero,5.10a,18th pitch of the day and 32 of 2 days haha,8,Nevada > Southern Nevada > La Madre Range > La...,3.2,2,Lead,Onsight,...,True,1,1,True,True,False,Lake Mead Buttress,La Madre Range,Lake Mead Buttress,5.10
8,2024-12-22,Contagion,5.10a/b,In 3 pitches - fun! I linked first 2.5 with th...,6,Nevada > Southern Nevada > La Madre Range > La...,3.5,3,Lead,Onsight,...,True,1,1,True,True,False,Lake Mead Buttress,La Madre Range,Lake Mead Buttress,5.10
9,2024-12-22,Killer Nano Robot Infection,5.12a,One fall at first perma :( just simply didn’t ...,4,Nevada > Southern Nevada > La Madre Range > La...,4.0,4,Lead,Redpoint,...,True,1,1,False,True,False,Lake Mead Buttress,La Madre Range,Lake Mead Buttress,5.12


In [15]:
df.describe()

,Date,Pitches,Avg Stars,Your Stars,Length,Rating Code,Year,Month,Session_ID,Days_Attempted
count,2163,2163.000000,2163.000000,2163.000000,1921.000000,2163.000000,2163.000000,2163.000000,2163.000000,2163.000000
mean,2022-08-15 21:44:11.317614336,1.367545,3.044799,2.460934,120.087975,6670.727231,2022.104485,6.734628,1.161350,2.409616
min,2018-07-28 00:00:00,1.000000,0.600000,-1.000000,8.000000,0.000000,2018.000000,1.000000,1.000000,1.000000
25%,2021-08-16 00:00:00,1.000000,2.600000,2.000000,60.000000,2400.000000,2021.000000,4.000000,1.000000,1.000000
50%,2022-10-06 00:00:00,1.000000,3.100000,3.000000,80.000000,3500.000000,2022.000000,7.000000,1.000000,2.000000
75%,2023-12-26 00:00:00,1.000000,3.600000,3.000000,100.000000,6600.000000,2023.000000,10.000000,1.000000,3.000000
max,2024-12-24 00:00:00,20.000000,4.000000,4.000000,10700.000000,32250.000000,2024.000000,12.000000,4.000000,11.000000
std,NaN,1.375550,0.592572,1.142779,301.584814,6607.580219,1.539548,3.341057,0.403895,1.982938


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2163 entries, 0 to 2162
Data columns (total 38 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            2163 non-null   datetime64[ns]
 1   Route           2163 non-null   object        
 2   Rating          2163 non-null   object        
 3   Notes           2163 non-null   object        
 4   Pitches         2163 non-null   int64         
 5   Location        2163 non-null   object        
 6   Avg Stars       2163 non-null   float64       
 7   Your Stars      2163 non-null   int64         
 8   Style           2141 non-null   object        
 9   Lead Style      1439 non-null   object        
 10  Route Type      2163 non-null   object        
 11  Your Rating     740 non-null    object        
 12  Length          1921 non-null   float64       
 13  Rating Code     2163 non-null   int64         
 14  Country         2163 non-null   object        
 15  Stat

### Write Engineered & Processed Data to Google Drive for use in other Analytical Applications. I.E. Tableau

In [17]:
# Save the DataFrame to CSV in the same Google Drive folder
output_path = "/content/drive/My Drive/Colab Notebooks/personal_projects/climbing_analysis/processed_climbing_data.csv"
df.to_csv(output_path, index=False)

print(f"Data saved to: {output_path}")

Data saved to: /content/drive/My Drive/Colab Notebooks/personal_projects/climbing_analysis/processed_climbing_data.csv


###Call functions to view climbing metrics

In [18]:
df['Rating'] = df['Rating'].apply(clean_grade)
df = df.dropna(subset=['Rating'])

# Dropping Airstream because I did not send the 5.13a pitch. I only climbed through the first 2 pitches
df = df[df['Route'] != 'Airstream']
analyze_climbing_metrics(df)

{'total_climbs': 1763,
 'unique_routes': 1047,
 'avg_stars': 2.526942711287578,
 'total_pitches': 2479,
 'sport_climbs': 1366,
 'trad_climbs': 404,
 'boulder_problems': 5,
 'multipitch_climbs': 213,
 'sport_success_rate': 66.0,
 'trad_success_rate': 63.1,
 'boulder_success_rate': 100.0,
 'onsight_count': 699,
 'flash_count': 1,
 'redpoint_count': 385,
 'fell_hung_count': 254,
 'total_projects': 231,
 'average_attempts_per_route': 2.2,
 'hardest_sport': '5.13a',
 'hardest_sport_clean': '5.12d',
 'hardest_trad': '5.12c',
 'hardest_trad_clean': '5.12a',
 'hardest_boulder': '5.10c',
 'hardest_boulder_clean': '5.10c',
 'unique_areas': 72,
 'unique_crags': 341,
 'most_visited_area': 'Bishop Area',
 'most_visited_crag': 'Mustache Wall',
 'grade_distribution': {'5.9 and under': 586,
  '5.10': 558,
  '5.11': 350,
  '5.12': 253,
  '5.13': 11,
  'Boulder': 5},
 'climbing_days': 524,
 'weekend_ratio': 44.9,
 'season_distribution': {'Summer': 526,
  'Fall': 517,
  'Spring': 443,
  'Winter': 277},
 

### Most Climbed Locations

In [19]:
get_ticks_by_crag(df).head(25)

,count
Crag,
Mustache Wall,82
Gong Show Wall,51
Banana Belt,47
Planetarium / Gecko Wall,46
Great Wall of China,45
Turkey Perch,37
Faulty Tower,32
Pub Wall,30
Diamondback Area,27


In [20]:
get_ticks_by_country(df).head(25)

,count
Country,
United States,1655
Spain,71
Greece,37


### Hardest Routes & Boulders by Week

In [21]:
hardest_roped_routes_by_week = get_hardest_climb_by_week(df[df['Is_Boulder'] == False])
hardest_roped_routes_by_week.tail(10)

,Week,Number_of_Climbs,Hardest_Route,Grade,Style
297,2024-10-27,4,Orange Juice,5.12c,Fell/Hung
298,2024-11-03,3,Demon Seed,5.12c,Redpoint
299,2024-11-10,0,None,None,None
300,2024-11-17,5,Unnamed Route 46,5.11d,Onsight
301,2024-11-24,3,Penélope Cruz,5.12d,Fell/Hung
302,2024-12-01,1,Avance Cangrejo,5.12d,Fell/Hung
303,2024-12-08,0,None,None,None
304,2024-12-15,2,Fight or Flight,5.12d,Fell/Hung
305,2024-12-22,2,Killer Nano Robot Infection,5.12a,Redpoint
306,2024-12-29,2,"Fear and Loathing, III",5.12a,Fell/Hung


In [22]:
hardest_boulders_by_week = get_hardest_climb_by_week(df[df['Is_Boulder'] == True])
hardest_boulders_by_week = hardest_boulders_by_week.rename(columns={'Hardest_Route': 'Hardest_Boulder'})
hardest_boulders_by_week = hardest_boulders_by_week.rename(columns={'Number_of_Climbs': 'Number_of_Boulders'})
hardest_boulders_by_week.tail(10)

,Week,Number_of_Boulders,Hardest_Boulder,Grade,Style
143,2022-09-04,0,None,None,None
144,2022-09-11,0,None,None,None
145,2022-09-18,0,None,None,None
146,2022-09-25,0,None,None,None
147,2022-10-02,0,None,None,None
148,2022-10-09,0,None,None,None
149,2022-10-16,0,None,None,None
150,2022-10-23,0,None,None,None
151,2022-10-30,0,None,None,None
152,2022-11-06,1,Olmsted Crack,5.9,None


### Quick Look at some of my projects

In [23]:
tmp = df[df['Is_Project'] == True]
tmp.head(25)

,Date,Route,Rating,Notes,Pitches,Location,Avg Stars,Your Stars,Style,Lead Style,...,Is_Weekend,Session_ID,Days_Attempted,Was_First_Try,Is_Success,Is_Project,Crag,Area,Subarea,Grade_Category
32,2024-11-21,Mandragora,5.12c,"Maybe not as hard, but I think better than Ora...",1,International > Europe > Spain > Catalonia > S...,3.9,4,Lead,Redpoint,...,False,1,2,False,True,True,Espero Primavera,Spain,Catalonia,5.12
33,2024-11-21,Mandragora,5.12c,"1 fall, 1 hang. First climb of the day",1,International > Europe > Spain > Catalonia > S...,3.9,4,Lead,Fell/Hung,...,False,2,2,False,False,True,Espero Primavera,Spain,Catalonia,5.12
53,2024-11-02,Breakfast Burrito,5.10c,World class,1,Kentucky > Red River Gorge > Pendergrass-Murra...,3.7,4,Lead,Redpoint,...,True,1,4,False,True,True,Drive-By Crag,Pendergrass-Murray Recreational Preserve (PMRP),Drive-By Crag,5.10
55,2024-11-02,Demon Seed,5.12c,Well - I think between this and Ghostrider in ...,1,Kentucky > Red River Gorge > Pendergrass-Murra...,3.9,4,Lead,Redpoint,...,True,1,2,False,True,True,Bob Marley,Pendergrass-Murray Recreational Preserve (PMRP),Bob Marley,5.12
56,2024-11-02,Demon Seed,5.12c,Actually used strategy lol. Bolt to bolted and...,1,Kentucky > Red River Gorge > Pendergrass-Murra...,3.9,4,Lead,Fell/Hung,...,True,2,2,False,False,True,Bob Marley,Pendergrass-Murray Recreational Preserve (PMRP),Bob Marley,5.12
61,2024-11-01,The Force,5.13a,First go - going to anchors; wasn’t warmed up\...,2,Kentucky > Red River Gorge > Pendergrass-Murra...,3.9,4,Lead,Fell/Hung,...,False,1,5,False,False,True,The Dark Side,Pendergrass-Murray Recreational Preserve (PMRP),The Dark Side,5.13
74,2024-10-24,Demon Seed,5.12c,1 fall :( past all the cruxes REALLY fought. W...,1,Kentucky > Red River Gorge > Pendergrass-Murra...,3.9,4,Lead,Fell/Hung,...,False,1,2,False,False,True,Bob Marley,Pendergrass-Murray Recreational Preserve (PMRP),Bob Marley,5.12
75,2024-10-24,Demon Seed,5.12c,Onsight attempt. Horrible skin. Tired\n\n1 fal...,1,Kentucky > Red River Gorge > Pendergrass-Murra...,3.9,4,Lead,Fell/Hung,...,False,2,2,False,False,True,Bob Marley,Pendergrass-Murray Recreational Preserve (PMRP),Bob Marley,5.12
79,2024-10-23,The Force,5.13a,Time for a break. Sent through the crux again....,1,Kentucky > Red River Gorge > Pendergrass-Murra...,3.9,4,Lead,Fell/Hung,...,False,1,5,False,False,True,The Dark Side,Pendergrass-Murray Recreational Preserve (PMRP),The Dark Side,5.13
81,2024-10-20,The Force,5.13a,Couldn’t be closer! Fell going to the tooth. G...,1,Kentucky > Red River Gorge > Pendergrass-Murra...,3.9,4,Lead,Fell/Hung,...,True,1,5,False,False,True,The Dark Side,Pendergrass-Murray Recreational Preserve (PMRP),The Dark Side,5.13


### Transform object type columns to categorical for Visualization & Future ML purposes

In [24]:
object_cols = []
for col in df.columns:
    if df[col].dtype == 'object':
        object_cols.append(col)

df = convert_to_categorical(df, object_cols)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1763 entries, 0 to 2161
Data columns (total 38 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            1763 non-null   datetime64[ns]
 1   Route           1763 non-null   category      
 2   Rating          1763 non-null   category      
 3   Notes           1763 non-null   category      
 4   Pitches         1763 non-null   int64         
 5   Location        1763 non-null   category      
 6   Avg Stars       1763 non-null   float64       
 7   Your Stars      1763 non-null   int64         
 8   Style           1754 non-null   category      
 9   Lead Style      1409 non-null   category      
 10  Route Type      1763 non-null   category      
 11  Your Rating     696 non-null    category      
 12  Length          1637 non-null   float64       
 13  Rating Code     1763 non-null   int64         
 14  Country         1763 non-null   category      
 15  State    

## Data Visualization

### Imports

In [25]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

###Pitch Count by Crag

In [26]:
# Get the crag data
crag_counts = get_ticks_by_crag(df).head(15)

# Create the bar chart using plotly express for easier color gradient
fig = px.bar(
    x=crag_counts.index,
    y=crag_counts.values,
    color=crag_counts.values,  # Use count for color
    color_continuous_scale='viridis',  # Nice color gradient from purple to yellow
    title='Top 15 Most Visited Crags',
    labels={
        'x': 'Crag Name',
        'y': 'Number of Climbs',
        'color': 'Number of Climbs'
    }
)

# Update the layout
fig.update_layout(
    xaxis_tickangle=-45,  # Angle the crag names for better readability
    height=700,
    width=850,
    template='plotly_white',
    showlegend=False,
    xaxis={'categoryorder': 'total descending'}  # Sort bars descending
)

# Update color bar
fig.update_coloraxes(colorbar_title='Number of Ticks')

# Show the plot
fig.show()

### Unique Sends by Grade - No repeats
**NOTE** For grades with less than 15 sends, on mouse hover it will display all the names of the routes sent at that grade

In [27]:
# Only sends. Only routes
tmp = df[(df['Is_Success'] == True) & (df['Is_Boulder'] == False)]

# No repeats
tmp = tmp.drop_duplicates(subset='Route')

# Clean grades
tmp['Clean_Grade'] = tmp['Rating'].apply(clean_grade)
tmp = tmp[tmp['Clean_Grade'].notna()]  # Remove any None values

# Count sends by grade
grade_counts = tmp['Clean_Grade'].value_counts()

# Create a dictionary of routes for each grade
grade_routes = {}
for grade in grade_counts.index:
    routes = tmp[tmp['Clean_Grade'] == grade]['Route'].tolist()
    if grade_counts[grade] <= 15:  # Only store routes for grades with 5 or fewer sends
        grade_routes[grade] = '<br>'.join(sorted(routes))
    else:
        grade_routes[grade] = ''

# Create hover text
hover_text = [
    f"Grade: {grade}<br>Count: {count}<br><br>Routes:{grade_routes[grade]}"
    if count <= 15 else
    f"Grade: {grade}<br>Count: {count}"
    for grade, count in zip(grade_counts.index, grade_counts.values)
]

# Create the bar chart
fig = px.bar(
    x=grade_counts.index,
    y=grade_counts.values,
    color=grade_counts.values,
    color_continuous_scale='plasma',
    title='Send Distribution by Grade',
    labels={
        'x': 'Grade',
        'y': 'Number of Sends',
        'color': 'Number of Sends'
    }
)

# Update hover template
fig.update_traces(
    hovertemplate="%{customdata}<extra></extra>",
    customdata=hover_text
)

# Update the layout
fig.update_layout(
    height=600,
    width=1200,
    template='plotly_white',
    showlegend=False,
    bargap=0.1,
    hoverlabel=dict(
        bgcolor="white",
        font_size=14,
        align="left"
    )
)

# Update axes
fig.update_xaxes(title='Grade', tickangle=0)
fig.update_yaxes(title='Number of Sends')

# Show the plot
fig.show()

###Show Pitches by State

In [28]:
# Filter for US climbs
us_climbs = df[df['Country'] == 'United States'].copy()

# Create state abbreviation mapping
state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Add state abbreviations
us_climbs['State_Abbrev'] = us_climbs['State'].map(state_abbrev)

# Aggregate pitches by state
state_pitches = us_climbs.groupby('State_Abbrev')['Pitches'].sum().reset_index()

# Create base choropleth map
fig = px.choropleth(
    state_pitches,
    locations='State_Abbrev',
    locationmode='USA-states',
    color='Pitches',
    scope='usa',
    color_continuous_scale='RdBu',
    title='Total Climbing Pitches by State',
    labels={'Pitches': 'Number of Pitches'}
)

# Add text labels
fig.add_trace(go.Scattergeo(
    locations=state_pitches['State_Abbrev'],
    locationmode='USA-states',
    text=state_pitches['Pitches'].astype(int),
    mode='text',
    textfont=dict(color='white', size=10),
    showlegend=False
))

# Update layout
fig.update_layout(
    title_x=0.5,
    geo_scope='usa',
    width=1200,
    height=700,
    template='plotly_white'
)

# Show the map
fig.show()

### Pitches by Country

In [29]:
# Aggregate pitches by country
country_pitches = df.groupby('Country')['Pitches'].sum().reset_index()

# Create the choropleth map
fig = px.choropleth(
    country_pitches,
    locations='Country',
    locationmode='country names',
    color='Pitches',
    color_continuous_scale='Sunset',
    title='Total Climbing Pitches by Country',
    labels={'Pitches': 'Number of Pitches'},
    projection='equirectangular'  # You can also use 'equirectangular', 'mercator', 'orthographic'
)

# Update layout
fig.update_layout(
    title_x=0.5,
    width=1200,
    height=700,
    template='plotly_white',
    # Add margin to ensure title and color bar are visible
    margin=dict(r=20, t=40, l=20, b=20)
)

# Show the map
fig.show()

# Print summary statistics
print("\nPitches by Country:")
print(country_pitches.sort_values('Pitches', ascending=False))

<ipython-input-29-6e60505de81e>:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.




Pitches by Country:
         Country  Pitches
2  United States     2369
1          Spain       73
0         Greece       37


###Climbing by Month

In [32]:
import plotly.express as px
import pandas as pd

# Extract year and month from date
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

# Create month labels for better readability
month_labels = {
    1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr',
    5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug',
    9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}
df['Month_Name'] = df['Month'].map(month_labels)

# Aggregate climbing volume by year and month
volume_by_month = df.groupby(['Year', 'Month', 'Month_Name']).size().reset_index(name='Climbs')

# Create the heatmap
fig = px.density_heatmap(
    volume_by_month,
    x='Month_Name',
    y='Year',
    z='Climbs',
    color_continuous_scale='Viridis',
    title='Climbing Volume by Month and Year',
    labels={'Climbs': 'Number of Climbs', 'Month_Name': 'Month', 'Year': 'Year'},
    # text=volume_by_month['Climbs'].astype(int)  # Add text labels
)

# Update layout
fig.update_layout(
    height=1000,
    width=1000,
    template='plotly_white',
    title_x=0.5,
    # Ensure months are in correct order
    xaxis={'categoryarray': list(month_labels.values())},
    # Reverse year order so most recent is at top
    yaxis={'autorange': 'reversed'}
)

# Add text labels
fig.update_traces(
    texttemplate="%{z}",
    textfont={"size": 14}
)

# Show the plot
fig.show()

In [57]:
import plotly.graph_objects as go
import pandas as pd

# Filter for successful sends only
tmp['Clean_Grade'] = tmp['Rating'].apply(clean_grade)
sends_df = tmp[tmp['Is_Success'] == True].copy()

# Create clean categories for send style with Flash/Onsight combined
def categorize_style(row):
    if row['Is_Onsight'] or row['Is_Flash']:  # Combined Flash and Onsight
        return 'Flash/Onsight'
    elif row['Is_Redpoint']:
        return 'Redpoint'
    return 'Other'

sends_df['Send_Style'] = sends_df.apply(categorize_style, axis=1)

# Group by grade and style
grade_style_counts = sends_df.groupby(['Clean_Grade', 'Send_Style'], observed=True).size().reset_index(name='Count')

# Sort grades (keeping your existing grade_sort_key function)
all_grades = sorted(grade_style_counts['Clean_Grade'].unique(), key=grade_sort_key)

# Create separate dataframes for each style
flash_onsight_data = grade_style_counts[grade_style_counts['Send_Style'] == 'Flash/Onsight']
redpoint_data = grade_style_counts[grade_style_counts['Send_Style'] == 'Redpoint']

# Create the pyramid plot
fig = go.Figure()

# Add bars for each style
colors = {'Flash/Onsight': '#2ecc71', 'Redpoint': '#e74c3c'}

# Add Flash/Onsight bars (now showing positive values)
fig.add_trace(go.Bar(
    name='Flash/Onsight',
    y=all_grades,
    x=[flash_onsight_data[flash_onsight_data['Clean_Grade'] == grade]['Count'].iloc[0]
       if grade in flash_onsight_data['Clean_Grade'].values else 0
       for grade in all_grades],
    orientation='h',
    marker_color=colors['Flash/Onsight']
))

# Add redpoint bars
fig.add_trace(go.Bar(
    name='Redpoint',
    y=all_grades,
    x=[redpoint_data[redpoint_data['Clean_Grade'] == grade]['Count'].iloc[0]
       if grade in redpoint_data['Clean_Grade'].values else 0
       for grade in all_grades],
    orientation='h',
    marker_color=colors['Redpoint']
))

# Update layout
fig.update_layout(
    title='Climbing Grade Pyramid by Send Style',
    barmode='group',  # Changed to 'group' to show bars side by side
    yaxis={'title': 'Grade'},
    xaxis={'title': 'Number of Sends'},
    height=800,
    width=1000,
    template='plotly_white',
    showlegend=True,
    legend={'x': 0.5, 'y': 1.1, 'orientation': 'h'},
)

# Update hover template
fig.update_traces(
    hovertemplate="<b>%{y}</b><br>" +
                  "Sends: %{x}<br>" +
                  "<extra></extra>"
)

# Show the plot
fig.show()

# Print some statistics
print("\nGrade Distribution by Style:")
pivot_table = pd.pivot_table(
    grade_style_counts,
    values='Count',
    index='Clean_Grade',
    columns='Send_Style',
    fill_value=0,
    observed=True
).sort_index(key=lambda x: pd.Series(x).map(grade_sort_key))
print(pivot_table)


Grade Distribution by Style:
Send_Style   Flash/Onsight  Other  Redpoint
Clean_Grade                                
5.3                    0.0    1.0       0.0
5.4                    2.0    0.0       0.0
5.5                    2.0    0.0       0.0
5.6                   16.0    1.0       2.0
5.7                   43.0    2.0      16.0
5.8                   89.0    9.0      20.0
5.9                   98.0   10.0      36.0
5.10                  15.0    2.0       6.0
5.10a                 74.0    4.0      25.0
5.10b                 63.0    7.0      18.0
5.10c                 35.0    1.0      15.0
5.10d                 24.0    1.0      13.0
5.11a                 37.0    4.0      20.0
5.11                   4.0    1.0       3.0
5.11b                 26.0    7.0      12.0
5.11c                 14.0    3.0      14.0
5.11d                 16.0    3.0       5.0
5.12a                  7.0    4.0      23.0
5.12                   0.0    0.0       2.0
5.12b                  2.0    0.0      10.0
5.